In [1]:
# Things to keep in mind with the data:
# (1) The calendar week for Walmart changes each year, so the holiday week may capture more or less days related 
# to the holiday
# (2) The data proportionally has more weeks that are not holidays compared to weeks that are a holiday, this could possibly 
# skew our results
# (3) We are only looking at 4 major holidays, we are not including local holidays, sports events and other events
# that could possibly have an effect on weekly sales
# (4) This dataset only pulls from 45 stores out of all of Walmarts locations across the United States, this subset may not be 
# an accurate depiction of Walmart as a whole

In [2]:
#%run secondnotebook.ipynb

# Import necessary modules
import pandas as pd
import csv
from scipy import stats
import os

# Read in clean/transformed dataset
walmart_clean = pd.read_csv("Project 1\Walmart.csv")

# Display preview of dataframe
walmart_clean.head()

os.getcwd()

'C:\\Users\\phamr\\Desktop'

In [3]:
# Create a list identifying whether that week is a Holiday or No Holiday
is_holiday = []

for holiday in walmart_clean['Holiday Name']:
    
    if holiday == 'No Holiday':
        
        is_holiday.append("No Holiday")
    
    else:
        
        is_holiday.append("Holiday")
        
# Add new column equal to the list created        
walmart_clean['Is Holiday'] = is_holiday

# Group by Is Holiday
walmart_ttl = pd.DataFrame(walmart_clean.groupby('Is Holiday')['Weekly_Sales'].sum())

# Caclulate Percentage
walmart_ttl['Percentage %'] = walmart_ttl['Weekly_Sales']/walmart_ttl['Weekly_Sales'].sum()

# Format Weekly Sales and Percentage
walmart_ttl['Weekly_Sales'] = pd.DataFrame(walmart_ttl['Weekly_Sales'].map("${:,.2f}".format))
walmart_ttl['Percentage %'] = pd.DataFrame(walmart_ttl['Percentage %'].map("{:.0%}".format))

#Export
outputfile = "Project 1/Summary/total_percent.csv"
walmart_ttl.to_csv(outputfile, header = True)

# Add title to Dataframe
walmart_ttl = walmart_ttl.style.set_caption("Total Holiday Weekly Sales and Percentages").set_table_styles([{
    'selector': 'caption',
    'props': [('color', 'black'), ('font-size', '13px'), ('font-weight', 'bold')]
}])

# Display dataframe
walmart_ttl

## The 4 major holidays account for about 8% of total sales, this is not very high as the 4 holidays will account for
## 4 out of the 52 weeks in a year which is also a little less than 8% (4/52 = 7.69%)

,Weekly_Sales,Percentage %
Is Holiday,,
Holiday,"$534,564,028.89",8%
No Holiday,"$6,202,654,958.22",92%


In [4]:
# Mainly grouping by Date and Store, adding Holiday Name and Is Holiday to get the values
walmart_groupby = walmart_clean.groupby(['Date', 'Store','Holiday Name','Is Holiday'])

# Aggregate across our question variables, set Dataframe and reset index
walmart_holiday = pd.DataFrame(walmart_groupby.agg({'Weekly_Sales': 'sum', 
                                                    'Temperature': 'mean',
                                                   'CPI': 'mean',
                                                   'Unemployment': 'mean',
                                                   'Fuel_Price': 'mean'}).reset_index())

# Display preview of dataframe
walmart_holiday.head()

,Date,Store,Holiday Name,Is Holiday,Weekly_Sales,Temperature,CPI,Unemployment,Fuel_Price
0,2010-02-05,1,No Holiday,No Holiday,1643690.90,6,211.10,8.106,2.57
1,2010-02-05,2,No Holiday,No Holiday,2136989.46,5,210.75,8.324,2.57
2,2010-02-05,3,No Holiday,No Holiday,461622.22,8,214.42,7.368,2.57
3,2010-02-05,4,No Holiday,No Holiday,2135143.87,7,126.44,8.623,2.60
4,2010-02-05,5,No Holiday,No Holiday,317173.10,4,211.65,6.566,2.57


In [5]:
# Examining the weeks that have the highest Weekly Sales

walmart_holiday.sort_values(by= 'Weekly_Sales', ascending = False).head()

## Christmas has the top 3 highest weekly_sales out the 3 years in Walmart

,Date,Store,Holiday Name,Is Holiday,Weekly_Sales,Temperature,CPI,Unemployment,Fuel_Price
2083,2010-12-24,14,Christmas,Holiday,3818686.45,-1,182.54,8.724,3.14
2089,2010-12-24,20,Christmas,Holiday,3766687.43,-4,204.64,7.484,3.14
2079,2010-12-24,10,Christmas,Holiday,3749057.69,14,126.98,9.003,3.24
4413,2011-12-23,4,No Holiday,No Holiday,3676388.98,2,129.98,5.143,3.10
2082,2010-12-24,13,Christmas,Holiday,3595903.20,2,126.98,7.795,2.85


In [6]:
# Examining the weeks that have the lowest Weekly Sales

walmart_holiday.sort_values(by= 'Weekly_Sales', ascending = True).head()

## During the month of February had lowest/no sales during the 4 years

,Date,Store,Holiday Name,Is Holiday,Weekly_Sales,Temperature,CPI,Unemployment,Fuel_Price
7604,2013-04-26,45,No Holiday,No Holiday,0.0,10,193.59,8.335,3.62
7008,2013-01-25,34,No Holiday,No Holiday,0.0,7,132.09,8.934,3.22
7009,2013-01-25,35,No Holiday,No Holiday,0.0,-3,142.80,8.600,3.58
7010,2013-01-25,36,No Holiday,No Holiday,0.0,14,222.90,6.266,3.24
7011,2013-01-25,37,No Holiday,No Holiday,0.0,14,222.90,6.266,3.23


In [7]:
# (1) ************************************** TEMPERATURE **************************************
# (1A) What effect does temperature have on weekly sales? 
# Hypothesis: Expecting extreme weather temperature to have lower sales

# Create bin and label in dataframe, temperature increment of 10 degrees
bins = [ min(walmart_holiday['Temperature']) - 2, -10, -1, 9, 19, 29, max(walmart_holiday['Temperature']) + 2 ]
labels = [ "<-10°C", "-10°C to -1°C", "0°C to 9°C", "10°C to 19°C","20°C to 29°C", "≥30°C"]

# Add bin/label Column
walmart_holiday['Temp Category'] = pd.cut(walmart_holiday['Temperature'], bins, labels = labels)

# Display preview of dataframe
walmart_holiday.head()

,Date,Store,Holiday Name,Is Holiday,Weekly_Sales,Temperature,CPI,Unemployment,Fuel_Price,Temp Category
0,2010-02-05,1,No Holiday,No Holiday,1643690.90,6,211.10,8.106,2.57,0°C to 9°C
1,2010-02-05,2,No Holiday,No Holiday,2136989.46,5,210.75,8.324,2.57,0°C to 9°C
2,2010-02-05,3,No Holiday,No Holiday,461622.22,8,214.42,7.368,2.57,0°C to 9°C
3,2010-02-05,4,No Holiday,No Holiday,2135143.87,7,126.44,8.623,2.60,0°C to 9°C
4,2010-02-05,5,No Holiday,No Holiday,317173.10,4,211.65,6.566,2.57,0°C to 9°C


In [8]:
# (1A) TEMPERATURE CONT.

# Check Average Weekly Sale for Temperature Category
temp = pd.DataFrame(walmart_holiday.groupby('Temp Category')['Weekly_Sales'].mean())

# Format Weekly Sales
temp['Weekly_Sales'] = temp['Weekly_Sales'].map("${:,.2f}".format)

# Export File
outputfile = "Project 1/Temperature/temp_avg.csv"
temp.to_csv(outputfile, header = True)

# Add title
temp = temp.style.set_caption("Temperature vs. Average Weekly Sales").set_table_styles([{
    'selector': 'caption',
    'props': [('color', 'black'), ('font-size', '13px'), ('font-weight', 'bold')]
}])

# Display dataframe
temp

## Table suggests that weekly sales is a lot lower when the temperature is cold (< -10°C)
## When temperature is warm 20-29, average weekly sales is the highest

,Weekly_Sales
Temp Category,
<-10°C,"$402,754.59"
-10°C to -1°C,"$766,037.81"
0°C to 9°C,"$818,247.46"
10°C to 19°C,"$855,314.94"
20°C to 29°C,"$1,017,458.08"
≥30°C,"$894,782.07"


In [9]:
# (1A) TEMPERATURE CONT.

#Which temperature category has the highest total weekly sales
temp_ttl = pd.DataFrame(walmart_holiday.groupby("Temp Category")['Weekly_Sales'].sum())

# Calculate
temp_ttl['Percentage %'] = temp_ttl['Weekly_Sales']/temp_ttl['Weekly_Sales'].sum()

# Calculate Count
temp_ttl['Count'] = walmart_holiday.groupby("Temp Category")['Weekly_Sales'].count()

# Calculate Count Total
temp_ttl['Count Percentage %'] = temp_ttl['Count']/temp_ttl['Count'].sum()

# Format Weekly Sales
temp_ttl['Weekly_Sales'] = temp_ttl['Weekly_Sales'].map("${:,.2f}".format)
temp_ttl['Percentage %'] = temp_ttl['Percentage %'].map("{:.1%}".format)
temp_ttl['Count Percentage %'] = temp_ttl['Count Percentage %'].map("{:.1%}".format)
                        
# Export file
outputfile = "Project 1/Temperature/temp_ttl.csv"
temp_ttl.to_csv(outputfile, header = True)

# Add title
temp_ttl = temp_ttl.style.set_caption("Temperature vs. Total Weekly Sales").set_table_styles([{
    'selector': 'caption',
    'props': [('color', 'black'), ('font-size', '13px'), ('font-weight', 'bold')]
}])
                        
# Display dataframe
temp_ttl

## Table suggests that weekly sales is a lot lower when the temperature is cold (< -10°C)
## When temperature is warm 20-29, total weekly sales is the highest accounting for 1/3 of total revenue

,Weekly_Sales,Percentage %,Count,Count Percentage %
Temp Category,,,,
<-10°C,"$23,762,520.52",0.4%,59,0.8%
-10°C to -1°C,"$456,558,533.86",6.8%,596,7.8%
0°C to 9°C,"$1,467,117,694.38",21.8%,1793,23.6%
10°C to 19°C,"$2,063,874,961.30",30.6%,2413,31.7%
20°C to 29°C,"$2,244,512,522.11",33.3%,2206,29.0%
≥30°C,"$481,392,754.94",7.1%,538,7.1%


In [10]:
# (1A) TEMPERATURE CONT.

# Linear Regression
(slope, intercept, rvalue, pvalue, stderr) = stats.linregress(walmart_holiday['Temperature'], walmart_holiday["Weekly_Sales"])

# Calculate x-values and regression values
x_values = walmart_holiday['Temperature']
y_values = slope*x_values + intercept

# Create linear regression equation
line_eq = 'y = ' + str(round(slope,2)) + "x + " + str(round(intercept,2))

print(line_eq, round(rvalue,2))

## (1) Equation suggests a positive relationship between Weekly Sales and Temperature
## implying an increase in temperature is correlated to weekly sales
## (2) R-Value is very close to 0 which suggest a very bad fit to the data
## Linear regression is not a good model for the two variables

y = 7889.54x + 770623.02 0.13


In [11]:
# (1B) TEMPERATURE HOLIDAY

# (1B) What effect does temperature have on holiday weekly sales? 
# Hypothesis: Expecting extreme weather temperature to still have lower sales during the holiday

# Check Average Weekly Sale for Temperature Category if it's a holiday or not
temp_is = pd.DataFrame(walmart_holiday.groupby(["Is Holiday", "Temp Category"])["Weekly_Sales"].mean())

# Format Weekly Sales
temp_is['Weekly_Sales'] = temp_is['Weekly_Sales'].map("${:,.2f}".format)

# Export file
outputfile = "Project 1/Temperature/temp_avg_is.csv"
temp_is.to_csv(outputfile, header = True)

# Add title
temp_is = temp_is.style.set_caption("Temperature vs. Average Holiday/Regular Weekly Sales").set_table_styles([{
    'selector': 'caption',
   'props': [('color', 'black'), ('font-size', '13px'), ('font-weight', 'bold')]
}])

# Display dataframe
temp_is

## Surprisingly when it's the holiday and cold there's a bit more spending than expected until we get to -10,
## possibly explanation is Christmas

In [12]:
# (1B) TEMPERATURE HOLIDAY CONT.

#Take Subset Dataframe of Holiday and Not Holiday
not_holiday = walmart_holiday.loc[walmart_holiday['Is Holiday'] == 'No Holiday', :]
holiday = walmart_holiday.loc[walmart_holiday['Is Holiday'] == 'Holiday', :]

#Calculate linear regression coefficients
(slope_h, intercept_h, rvalue_h, pvalue_h, stderr_h) = stats.linregress(holiday['Temperature'], holiday["Weekly_Sales"])
(slope, intercept, rvalue, pvalue, stderr) = stats.linregress(not_holiday['Temperature'], not_holiday["Weekly_Sales"])

#Calculate x-value and regression values
x_holiday = holiday['Temperature']
y_holiday = slope_h*x_holiday + intercept_h
x_values = not_holiday['Temperature']
y_values = slope*x_values + intercept

#Create linear regression equation
eq_holiday = 'y = ' + str(round(slope_h,2)) + "x + " + str(round(intercept_h,2))
eq_regular = 'y = ' + str(round(slope,2)) + "x + " + str(round(intercept,2))

print("Temperature Holiday:",eq_holiday,"R-value:", rvalue_h)
print("Temperature No Holiday:", eq_regular, "R-value:", rvalue)

## After subsetting data, linear regression are both not a good model to fit the two variables
## Positive correlation between temperature and weekly sales during holiday and not during holiday

Temperature Holiday: y = 9131.11x + 824143.69 R-value: 0.13059073194189183
Temperature No Holiday: y = 8033.54x + 762985.85 R-value: 0.13073203583185242


In [13]:
# (2) ************************************** FUEL PRICES **************************************
# (2A) What effect does regional fuel prices have on weekly sales?
# Hypothesis: Would expect higher fuel prices would cause a decrease in weekly sales, since less money to spend

#Linear Regression
(slope, intercept, rvalue, pvalue, stderr) = stats.linregress(walmart_holiday['Fuel_Price'], walmart_holiday["Weekly_Sales"])

x_values = walmart_holiday['Fuel_Price']
y_values = slope*x_values + intercept

line_eq = 'y = ' + str(round(slope,2)) + "x + " + str(round(intercept,2))

print(line_eq, round(rvalue,2))

## (1) Negative relationship which suggests that an increase in fuel prices results in a decrease in weekly sales
## (2) R-value is close to 0 suggesting linear regression is not a good fit for data

y = -130443.65x + 1328116.73 -0.09


In [14]:
# (2B) FUEL PRICES HOLIDAY
# (2B) What effect does regional fuel prices have on holiday weekly sales?
# Hypothesis: Would expect higher fuel prices would cause a decrease in weekly sales, since less money to spend

# Group by Holiday Name and take average of fuel price and weekly sales
fuel_avg = pd.DataFrame(walmart_holiday.groupby('Holiday Name')[['Fuel_Price','Weekly_Sales']].mean())

# Format Weekly Sales
fuel_avg['Fuel_Price'] = fuel_avg['Fuel_Price'].map("${:,.2f}".format)
fuel_avg['Weekly_Sales'] = fuel_avg['Weekly_Sales'].map("${:,.2f}".format)

# Export file
outputfile = "Project 1/Fuel Price/fuel_avg.csv"
fuel_avg.to_csv(outputfile, header = True)

# Add title
fuel_avg = fuel_avg.style.set_caption("Average Fuel Price vs. Weekly Sales During Holidays").set_table_styles([{
    'selector': 'caption',
   'props': [('color', 'black'), ('font-size', '13px'), ('font-weight', 'bold')]
}])

fuel_avg

,Fuel_Price,Weekly_Sales
Holiday Name,,
Christmas,$3.17,"$930,035.53"
Independence Day,$3.29,"$1,088,976.41"
New Year,$3.24,"$649,860.81"
No Holiday,$3.40,"$883,569.08"
Thanksgiving,$3.29,"$980,848.95"


In [15]:
# (2B) FUEL PRICES HOLIDAY CONT.

# Group by Is Holiday and take average of fuel price and weekly sales
fuel_is = pd.DataFrame(walmart_holiday.groupby('Is Holiday')[['Fuel_Price','Weekly_Sales']].mean())

# Format Weekly Sales
fuel_is['Fuel_Price'] = fuel_is['Fuel_Price'].map("${:,.2f}".format)
fuel_is['Weekly_Sales'] = fuel_is['Weekly_Sales'].map("${:,.2f}".format)

# Export file
outputfile = "Project 1/Fuel Price/fuel_avg_is.csv"
fuel_is.to_csv(outputfile, header = True)

# Add title
fuel_is = fuel_is.style.set_caption("Average Fuel Price vs. Regular/Holiday Weekly Sales").set_table_styles([{
    'selector': 'caption',
   'props': [('color', 'black'), ('font-size', '13px'), ('font-weight', 'bold')]
}])

fuel_is

,Fuel_Price,Weekly_Sales
Is Holiday,,
Holiday,$3.24,"$913,784.66"
No Holiday,$3.40,"$883,569.08"


In [16]:
# (2B) FUEL PRICES CONT.

# Calculate linear regression coefficients
(slope_h, intercept_h, rvalue_h, pvalue_h, stderr_h) = stats.linregress(holiday['Fuel_Price'], holiday["Weekly_Sales"])
(slope, intercept, rvalue, pvalue, stderr) = stats.linregress(not_holiday['Fuel_Price'], not_holiday["Weekly_Sales"])

# Calculate x-value and regression values
x_holiday = holiday['Fuel_Price']
y_holiday = slope_h*x_holiday + intercept_h
x_values = not_holiday['Fuel_Price']
y_values = slope*x_values + intercept

# Create linear regression equation
eq_holiday = 'y = ' + str(round(slope_h,2)) + "x + " + str(round(intercept_h,2))
eq_regular = 'y = ' + str(round(slope,2)) + "x + " + str(round(intercept,2))

print("Fuel Price Holiday: ",eq_holiday,"R-value:", round(rvalue_h,2))
print("Fuel Price No Holiday:",eq_regular,"R-value:", round(rvalue,2))

## Negative correlation between fuel price and weekly sales, not surprisingly as fuel prices increases, weekly sales decreases
## since families would have to spend more 

Fuel Price Holiday:  y = -435215.53x + 2325356.01 R-value: -0.17
Fuel Price No Holiday: y = -117757.17x + 1284223.6 R-value: -0.08


In [17]:
# (3) ************************************** UNEMPLOYMENT RATE **************************************
# (3A) What effect does the regional unemployment rate have on weekly sales?
# Hypothesis: Would expect lower weekly sales with a higher unemployemnt rate in the area

# Categorize unemployment
bins = [min(walmart_holiday['Unemployment']) - 2, 6, 9, max(walmart_holiday['Unemployment']) + 2]
labels = ["Low: <6%", "Normal: 6%-9%" , "High: >9%"]

# Add bin/label to dataframe
walmart_holiday['Unemployment Category'] = pd.cut(walmart_holiday['Unemployment'], bins, labels = labels)

# Display preview of dataframe
walmart_holiday.head()

,Date,Store,Holiday Name,Is Holiday,Weekly_Sales,Temperature,CPI,Unemployment,Fuel_Price,Temp Category,Unemployment Category
0,2010-02-05,1,No Holiday,No Holiday,1643690.90,6,211.10,8.106,2.57,0°C to 9°C,Normal: 6%-9%
1,2010-02-05,2,No Holiday,No Holiday,2136989.46,5,210.75,8.324,2.57,0°C to 9°C,Normal: 6%-9%
2,2010-02-05,3,No Holiday,No Holiday,461622.22,8,214.42,7.368,2.57,0°C to 9°C,Normal: 6%-9%
3,2010-02-05,4,No Holiday,No Holiday,2135143.87,7,126.44,8.623,2.60,0°C to 9°C,Normal: 6%-9%
4,2010-02-05,5,No Holiday,No Holiday,317173.10,4,211.65,6.566,2.57,0°C to 9°C,Normal: 6%-9%


In [18]:
# (3A) UNEMPLOYMENT CONT.

# Check Average Weekly Sale for Unemployment Category
rate = pd.DataFrame(walmart_holiday.groupby('Unemployment Category')['Weekly_Sales'].mean())

# Format Weekly Sales
rate['Weekly_Sales'] = rate['Weekly_Sales'].map("${:,.2f}".format)

# Export file
outputfile = "Project 1/Unemployment/rate_avg.csv"
rate.to_csv(outputfile, header = True)

# Add title
rate = rate.style.set_caption("Unemployment vs. Average Weekly Sales").set_table_styles([{
    'selector': 'caption',
    'props': [('color', 'black'), ('font-size', '13px'), ('font-weight', 'bold')]
}])

# Display dataframe
rate

## Not surprisingly, from table it suggest when unemployment rate is lower the average weekly sales is higher
## and vice versa, when unemployment rate is high, average weekly sales is lower

,Weekly_Sales
Unemployment Category,
Low: <6%,"$787,068.73"
Normal: 6%-9%,"$923,509.20"
High: >9%,"$793,263.48"


In [19]:
# Check Total Weekly Sale for Unemployment Category
rate_ttl = pd.DataFrame(walmart_holiday.groupby('Unemployment Category')['Weekly_Sales'].sum())

# Calculate Percent
rate_ttl['Percentage %'] = rate_ttl['Weekly_Sales']/rate_ttl['Weekly_Sales'].sum()

#Calculate Count 
rate_ttl['Count'] = walmart_holiday.groupby('Unemployment Category')['Weekly_Sales'].count()

# Calculate Percent
rate_ttl['Count Percentage %'] = rate_ttl['Count']/rate_ttl['Count'].sum()

# Format Weekly Sales
rate_ttl['Weekly_Sales'] = rate_ttl['Weekly_Sales'].map("${:,.2f}".format)
rate_ttl['Percentage %'] = rate_ttl['Percentage %'].map("{:.0%}".format)
rate_ttl['Count Percentage %'] = rate_ttl['Count Percentage %'].map("{:.0%}".format)

# Export file
outputfile = "Project 1/Unemployment/rate_ttl.csv"
rate_ttl.to_csv(outputfile, header = True)

# Add title
rate_ttl = rate_ttl.style.set_caption("Unemployment vs. Total Weekly Sales").set_table_styles([{
    'selector': 'caption',
    'props': [('color', 'black'), ('font-size', '13px'), ('font-weight', 'bold')]
}])

# Display dataframe
rate_ttl

## Most of the sales occur when unemployment rate is normal

,Weekly_Sales,Percentage %,Count,Count Percentage %
Unemployment Category,,,,
Low: <6%,"$734,335,122.95",11%,933,12%
Normal: 6%-9%,"$5,035,895,676.42",75%,5453,72%
High: >9%,"$966,988,187.74",14%,1219,16%


In [20]:
# (3A) UNEMPLOYMENT CONT.

unemployment = walmart_holiday

# Linear Regression
(slope, intercept, rvalue, pvalue, stderr) = stats.linregress(unemployment['Unemployment'], unemployment["Weekly_Sales"])

x_values = unemployment['Unemployment']
y_values = slope*x_values + intercept

line_eq = 'y = ' + str(round(slope,2)) + "x + " + str(round(intercept,2))
print(line_eq, rvalue)

## Positive correlation between unemployment rate and weekly sales, not surprising, higher unemployment rate
## results in higher weekly sales.
## R-value is not high so linear regression is not a good model fit for this data

y = 16337.38x + 758023.6 0.047768966800300273


In [21]:
# (3B) UNEMPLOYMENT HOLIDAY
# (3B) What effect does the regional unemployment rate have on holiday weekly sales?
# Hypothesis: Would expect lower weekly sales with a higher unemployemnt rate in the area even during holidays

# Check Average Weekly Sale for Unemployment Category
rate_is = pd.DataFrame(walmart_holiday.groupby(['Is Holiday','Unemployment Category'])['Weekly_Sales'].mean())

# Format Weekly Sales
rate_is['Weekly_Sales'] = rate_is['Weekly_Sales'].map("${:,.2f}".format)

# Export file
outputfile = "Project 1/Unemployment/rate_avg_is.csv"
rate_is.to_csv(outputfile, header = True)

# Add title
rate_is = rate_is.style.set_caption("Unemployment vs. Average Regular/Holiday Weekly Sales").set_table_styles([{
    'selector': 'caption',
    'props': [('color', 'black'), ('font-size', '13px'), ('font-weight', 'bold')]
}])

# Display dataframe
rate_is

## Intereseting results, when unemployemnts rate is high or normal, weekly sales is higher during the holidays
## and lower when unmployement rate is low.
## When it is not the holiday it has normal results where low unemployment rate has higher spending and vice versa

In [22]:
# (3B) UNEMPLOYMENT HOLIDAY CONT.

# Check Average Weekly Sale for Unemployment Category
rate_holiday = pd.DataFrame(walmart_holiday.groupby(['Holiday Name','Unemployment Category'])['Weekly_Sales'].mean())

# Format Weekly Sales
rate_holiday['Weekly_Sales'] = rate_holiday['Weekly_Sales'].map("${:,.2f}".format)

# Export file
outputfile = "Project 1/Unemployment/rate_avg_holiday.csv"
rate_holiday.to_csv(outputfile, header = True)

# Add title
rate_holiday = rate_holiday.style.set_caption("Unemployment vs. Average Weekly Sales During Holiday").set_table_styles([{
    'selector': 'caption',
    'props': [('color', 'black'), ('font-size', '13px'), ('font-weight', 'bold')]
}])

# Display dataframe
rate_holiday

## Surprisingly during Christmas average weekly_sales is even more when unemployemnt is very high or normal 
## compared to when it's low 

In [23]:
# (3B) UNEMPLOYMENT HOLIDAY CONT.

# Calculate linear regression coefficients
(slope_h, intercept_h, rvalue_h, pvalue_h, stderr_h) = stats.linregress(holiday['Unemployment'], holiday["Weekly_Sales"])
(slope, intercept, rvalue, pvalue, stderr) = stats.linregress(not_holiday['Unemployment'], not_holiday["Weekly_Sales"])

# Calculate x-value and regression values
x_holiday = holiday['Unemployment']
y_holiday = slope_h*x_holiday + intercept_h
x_values = not_holiday['Unemployment']
y_values = slope*x_values + intercept

# Create linear regression equation
eq_holiday = 'y = ' + str(round(slope_h,2)) + "x + " + str(round(intercept_h,2))
eq_regular = 'y = ' + str(round(slope,2)) + "x + " + str(round(intercept,2))

print("Unemployment Holiday: ",eq_holiday,"R-value:", round(rvalue_h,2))
print("Unemployment No Holiday:",eq_regular,"R-value:", round(rvalue,2))

## Surprising correlation, as unemployment rate get higher weekly sales get higher
## R-value is low, linear regression is not a good fit for data

Unemployment Holiday:  y = 43747.44x + 572618.21 R-value: 0.1
Unemployment No Holiday: y = 14058.93x + 773499.26 R-value: 0.04


In [24]:
# (4) ************************************** CPI RATE **************************************
# (4A) What effect does CPI have on weekly sales?
# Hypothesis: Would expect lower weekly sales with a higher CPI as it'll cost more to maintain the same standard of living

cpi = walmart_holiday 

# Linear Regression
(slope, intercept, rvalue, pvalue, stderr) = stats.linregress(cpi['CPI'], cpi["Weekly_Sales"])

# Calculate x_value and regression values
x_values = cpi['CPI']
y_values = slope*x_values + intercept

# Create equation
line_eq = 'y = ' + str(round(slope,2)) + "x + " + str(round(intercept,2))
print(line_eq, rvalue)

## Negative correlation between CPI and weekly sales, not surprising, higher CPI results in 
## a lower weekly sales, since with a higher CPI as it'll cost more to maintain the same standard of living.
## R-value is not high so linear regression is a good model fit for this data

y = -1359.58x + 1120367.03 -0.08414951460453712


In [25]:
# (4B) CPI HOLIDAY
# (4B) What effect does CPI have on weekly sales during holidays?
# Hypothesis: Would expect lower weekly sales with a higher CPI as it'll cost more to maintain the same standard of living

# Groupby Is Holiday and create dataframe
cpi_is = pd.DataFrame(walmart_holiday.groupby('Is Holiday')[['CPI','Weekly_Sales']].mean())

# Format weekly sales
cpi_is['Weekly_Sales'] = cpi_is['Weekly_Sales'].map("${:,.2f}".format)

# Export file
outputfile = "Project 1/CPI/cpi_is_avg.csv"
cpi_is.to_csv(outputfile, header = True)

# Add title
cpi_is = cpi_is.style.set_caption("CPI vs. Average Regular/Holiday Weekly Sales").set_table_styles([{
    'selector': 'caption',
    'props': [('color', 'black'), ('font-size', '13px'), ('font-weight', 'bold')]
}])

# Display dataframe
cpi_is

,CPI,Weekly_Sales
Is Holiday,,
Holiday,172.426701,"$913,784.66"
No Holiday,172.463764,"$883,569.08"


In [26]:
# (4B) CPI HOLIDAY CONT.

# Groupby Is Holiday and create dataframe
cpi_holiday = pd.DataFrame(walmart_holiday.groupby('Holiday Name')[['CPI', 'Weekly_Sales']].mean())

# Format weekly sales
cpi_holiday['Weekly_Sales'] = cpi_holiday['Weekly_Sales'].map("${:,.2f}".format)

# Export file
outputfile = "Project 1/CPI/cpi_holiday_avg.csv"
cpi_holiday.to_csv(outputfile, header = True)

# Add title
cpi_holiday = cpi_holiday.style.set_caption("CPI vs. Average Weekly Sales During Holidays").set_table_styles([{
    'selector': 'caption',
    'props': [('color', 'black'), ('font-size', '13px'), ('font-weight', 'bold')]
}])

# Display dataframe
cpi_holiday

,CPI,Weekly_Sales
Holiday Name,,
Christmas,171.999611,"$930,035.53"
Independence Day,171.697111,"$1,088,976.41"
New Year,173.272222,"$649,860.81"
No Holiday,172.463764,"$883,569.08"
Thanksgiving,172.880222,"$980,848.95"


In [27]:
# (4) CPI HOLIDAY CONT.

# Calculate linear regression coefficients
(slope_h, intercept_h, rvalue_h, pvalue_h, stderr_h) = stats.linregress(holiday['CPI'], holiday["Weekly_Sales"])
(slope, intercept, rvalue, pvalue, stderr) = stats.linregress(not_holiday['CPI'], not_holiday["Weekly_Sales"])

# Calculate x-value and regression values
x_holiday = holiday['CPI']
y_holiday = slope_h*x_holiday + intercept_h
x_values = not_holiday['CPI']
y_values = slope*x_values + intercept

# Create linear regression equation
eq_holiday = 'y = ' + str(round(slope_h,2)) + "x + " + str(round(intercept_h,2))
eq_regular = 'y = ' + str(round(slope,2)) + "x + " + str(round(intercept,2))

print("CPI Holiday: ",eq_holiday,"R-value:", round(rvalue_h,2))
print("CPI No Holiday:",eq_regular,"R-value:", round(rvalue,2))

## Not surprising correlation, increase in CPI cause a decrease in spending during holidays
## R-value is low, linear regression is not a good fit for data

CPI Holiday:  y = -1868.69x + 1235997.02 R-value: -0.09
CPI No Holiday: y = -1317.09x + 1110720.02 R-value: -0.08


In [28]:
# (5) ************************************** SUMMARY **************************************
# IS HOLIDAY SUMMARY
summary = walmart_holiday

# Aggregrate variables
isholiday_summary = summary.groupby('Is Holiday').agg({'Weekly_Sales': 'mean',
                                   'Temperature':'mean',
                                  'Fuel_Price': 'mean',
                                  'Unemployment': 'mean',
                                  'CPI':'mean'
                                  })

# Format columns
isholiday_summary['Weekly_Sales'] = pd.DataFrame(isholiday_summary['Weekly_Sales'].map("${:,.2f}".format))
isholiday_summary['Temperature'] = pd.DataFrame(isholiday_summary['Temperature'].map("{:,.0f} °C".format))
isholiday_summary['Fuel_Price'] = pd.DataFrame(isholiday_summary['Fuel_Price'].map("${:,.2f}".format))
isholiday_summary['Unemployment'] = pd.DataFrame(isholiday_summary['Unemployment'].map("{:,.2f}%".format))
isholiday_summary['CPI'] = pd.DataFrame(isholiday_summary['CPI'].map("{:,.2f}".format))

# Export file
outputfile = "Project 1/Summary/summary_is.csv"
isholiday_summary.to_csv(outputfile, header = True)

# Add title
isholiday_summary = isholiday_summary.style.set_caption("Is Holiday Summary Average").set_table_styles([{
    'selector': 'caption',
    'props': [('color', 'black'), ('font-size', '13px'), ('font-weight', 'bold')]
}])

# Display dataframe
isholiday_summary

,Weekly_Sales,Temperature,Fuel_Price,Unemployment,CPI
Is Holiday,,,,,
Holiday,"$913,784.66",10 °C,$3.24,7.80%,172.43
No Holiday,"$883,569.08",15 °C,$3.40,7.83%,172.46


In [29]:
# (5) SUMMARY CONT.
# HOLIDAY NAME SUMMARY

# Aggregrate variables
holiday_summary = summary.groupby('Holiday Name').agg({'Weekly_Sales': 'mean',
                                   'Temperature':'mean',
                                  'Fuel_Price': 'mean',
                                  'Unemployment': 'mean',
                                  'CPI':'mean'
                                  })

# Format columns
holiday_summary['Weekly_Sales'] = pd.DataFrame(holiday_summary['Weekly_Sales'].map("${:,.2f}".format))
holiday_summary['Temperature'] = pd.DataFrame(holiday_summary['Temperature'].map("{:,.0f} °C".format))
holiday_summary['Fuel_Price'] = pd.DataFrame(holiday_summary['Fuel_Price'].map("${:,.2f}".format))
holiday_summary['Unemployment'] = pd.DataFrame(holiday_summary['Unemployment'].map("{:,.2f}%".format))
holiday_summary['CPI'] = pd.DataFrame(holiday_summary['CPI'].map("{:,.2f}".format))

# Export
# Export file
outputfile = "Project 1/Summary/summary_holiday.csv"
holiday_summary.to_csv(outputfile, header = True)

# Add title
holiday_summary = holiday_summary.style.set_caption("Holiday Summary Average").set_table_styles([{
    'selector': 'caption',
    'props': [('color', 'black'), ('font-size', '13px'), ('font-weight', 'bold')]
}])

# Display dataframe
holiday_summary

,Weekly_Sales,Temperature,Fuel_Price,Unemployment,CPI
Holiday Name,,,,,
Christmas,"$930,035.53",3 °C,$3.17,7.93%,172.00
Independence Day,"$1,088,976.41",26 °C,$3.29,7.92%,171.70
New Year,"$649,860.81",2 °C,$3.24,7.56%,173.27
No Holiday,"$883,569.08",15 °C,$3.40,7.83%,172.46
Thanksgiving,"$980,848.95",9 °C,$3.29,7.74%,172.88


In [30]:
walmart_holiday.to_csv("Project 1/final_df.csv", header = True)
